In [1]:
import requests
import re
import os
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
import time
from typing import List, Dict, Set
import json

In [15]:
urls = [
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21.md",
  # "https://onestore-dev.gitbook.io/dev/tools/billing/old-version/v16.md",
  # "https://onestore-dev.gitbook.io/dev/tools/glb.md",
  # "https://onestore-dev.gitbook.io/dev/tools/billing/billing/v21/sdk.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/ov.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/pre.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/test.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/sdk.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.connectionstate.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.featuretype.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.producttype.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.recurringaction.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.responsecode.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchasedata.acknowledgestate.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchasedata.purchasestate.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchasedata.recurringstate.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseflowparams.prorationmode.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/acknowledgeparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/acknowledgeparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/consumeparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/consumeparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/iapresult.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/iapresult.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/productdetail.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/productdetailsparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/productdetailsparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/purchaseclient.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/purchaseclient.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/purchasedata.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/purchaseflowparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/purchaseflowparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/recurringproductparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/recurringproductparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/subscriptionparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/subscriptionparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/subscriptionupdateparams.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/classes/subscriptionupdateparams.builder.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/acknowledgelistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/consumelistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/iapresultlistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/productdetailslistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/purchaseclientstatelistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/purchaseslistener-deprecated.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/purchasesupdatedlistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/querypurchaseslistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/recurringproductlistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/interfaces/storeinfolistener.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/serverapi.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/pns.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/subs.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/releasenote.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/sample.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/upgrade.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/unity.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/unitymig.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/flutter.md",
  # "https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-9.md",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/unity",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-9",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-10",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-13",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-8",
  # "https://onestore-dev.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-12",
  # "https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-10.md",
  # "https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-13.md",
  # "https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-8.md",
  # "https://onestore-devs-organization.gitbook.io/guide-inapp-sdk/sdk-purchase-v21/flutter-sdk-v21#undefined-12.md",
  "https://onestore-dev.gitbook.io/dev/tools/billing/v21/web-payment.md"
]

In [17]:
merged_content = ""

# urls를 출력
for i, url in enumerate(urls, 1):
    print(f"processing #{i} : {url}")
    # url에서 content 추출
    response = requests.get(url)
    content = response.text
    # print(content)
    merged_content += f"출처: {url}\n"
    merged_content += content + "\n\n"
    time.sleep(1)
    
# 파일에 저장
with open(f"data/iap_gitbook_merged_all2.md", "w", encoding="utf-8") as f:
    f.write(merged_content)
    

    
    
    
    

processing #1 : https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
processing #2 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/ov.md
processing #3 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/pre.md
processing #4 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/test.md
processing #5 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/sdk.md
processing #6 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/references.md
processing #7 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations.md
processing #8 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.connectionstate.md
processing #9 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.featuretype.md
processing #10 : https://onestore-dev.gitbook.io/dev/tools/billing/v21/references/annotations/purchaseclient.producttype.md
processing #11 : https://onestore-dev.gitbook.io/dev/tools/billin